In [1]:
import pandas as pd
import numpy as np
import math

from sklearn.metrics import mean_squared_error

In [64]:
def norm(X):
    E = X.mean()
    D = X.std()
    X = (X-E)/D
    X['Post Promotion Status'] = 0
    x1 = pd.DataFrame({'feature 1': np.ones(nrows)})
    #print(X[X.columns[36:38]])
    result = pd.concat([x1,X], axis=1)
    return result

In [7]:
def normalize(X,Y,nrows):
    BinaryFeatures = ['Post Promotion Status','published_weekday_0','published_weekday_1','published_weekday_2',
                      'published_weekday_3', 'published_weekday_4','published_weekday_5','published_weekday_6',
                     'base_weekday_0','base_weekday_1','base_weekday_2','base_weekday_3','base_weekday_4',
                      'base_weekday_5','base_weekday_6']
    BF = X[BinaryFeatures] 
    
    DecimalFeatures = ['Page Popularity','Page Checkins','Page talking about',
                       'extra_0','extra_1','extra_2','extra_3','extra_4','extra_5',
                       'extra_6','extra_7','extra_8','extra_9','extra_10',
                       'extra_11','extra_12','extra_13','extra_14','extra_15','extra_16',
                       'extra_17','extra_18','extra_19','extra_20','extra_21','extra_22','extra_23','extra_24',
                       'CC1','CC2','CC3','CC4','CC5','Base Time','Post Length','Post Share Count','H Local']
    DF = X[DecimalFeatures]
    E = DF.mean()
    D = DF.std()
    DF = (DF-E)/D
    
    # 'H Local' feature's modification
    #HLocal = Y['Target']/X['H Local']#comments/an hour 
    #CategoryFeature =['Page Category']
    #CF = X[CategoryFeature]
    
    frequency = X['Page Category'].value_counts(sort=False) #frequency of categories
    CF = 1/frequency[X['Page Category']]
    CF = CF.reset_index(drop=True)
    
    # x0 = [1,1,...1] fpr b koeficient
    x1 = pd.DataFrame({'feature 1': np.ones(nrows)})

    result = pd.concat([x1,DF,CF, BF], axis=1) #,X['H Local']
    return result



In [8]:
def gradient(X,Y,W,lyambda, n, nfeat):
    Ypred = np.dot(X,W)
    #print(Y[:10], Ypred[:10])
    dy = Y - Ypred
    cost = np.sum(dy**2)/n
    #print( "cost", cost)
    gr = np.dot(X.T,dy)/n
    Wnew = W + 2* lyambda * gr.reshape(nfeat,1)
    return Wnew

def foo_RMSE(Xt, Yt, W):
    rmse = math.sqrt(np.sum(((Yt-np.dot(Xt,W))**2))/len(Yt))
    return rmse

def foo_R2(X,Y,W):
    Ey = Y.mean()
    r2 = 1 - np.sum((np.dot(X,W)-Y)**2)/np.sum((Y - Ey)**2)
    return r2

In [66]:
df = pd.read_csv('Dataset.csv', sep=',')
df = df.drop(df.columns[0],axis=1)
X = df[df.columns[:53]]
Y = df[['Target']]
X = normalize(X,Y,X.shape[0])
#X = norm(X)


In [74]:
#split into folds
nfolds = 5
indexes = np.arange(X.shape[0]//nfolds * nfolds)
indexes = indexes.reshape(X.shape[0]//nfolds,5)
each_folds_indexes =[] 
chunks_index = []
for i in range(nfolds):
    each_folds_indexes.append(indexes[:,i].tolist())
Xfold =[]
Yfold =[]
for ind in each_folds_indexes:
    Xfold.append(X.loc[ind])
    Yfold.append(Y.loc[ind])

#init   
nsteps = 400
e = 10e-4
lyambda = 0.05
table = pd.DataFrame({})
nfeatures = X.shape[1]
chunk = 0
for i in range(nfolds):
    
    #train/test split
    all_train_index=[] 
    for fold in range(nfolds):
       
        if fold!=i:
            all_train_index = all_train_index+each_folds_indexes[fold]
            
    Xtrain = X.loc[all_train_index]
    Ytrain = Y.loc[all_train_index]
    Xtest = X.loc[each_folds_indexes[i]] 
    Ytest = Y.loc[each_folds_indexes[i]]    
    
    nrows = Xtrain.shape[0]
    chunk+=1
    k = 0
    W = np.ones(nfeatures).reshape(nfeatures,1)
    
    while True:
        k+=1
        Wnew = gradient(Xtrain,Ytrain,W,lyambda/math.sqrt(k), nrows, nfeatures)
        estimation = np.max(abs(W - Wnew))
    
        if (estimation < e)or(k>nsteps):
            break
        W = Wnew
        
    print("CHUNK # ", chunk)    
    R2train = foo_R2(Xtrain,Ytrain,W)
    print ("R2 for train: ",R2train[0] , "\n")

    RMSEtrain = foo_RMSE(Xtrain,Ytrain,W)
    print ("RMSE for train: ",RMSEtrain , "\n") 
 
    
    #test
    R2test = foo_R2(Xtest,Ytest,W)
    print ("R2 for test: ",R2test[0] , "\n")

    RMSEtest = foo_RMSE(Xtest,Ytest,W)
    print ("RMSE for test: ",RMSEtest , "\n")
   
    
    a = [R2train[0],RMSEtrain,R2test[0],RMSEtest]
    
    for feature in W.tolist():
        a.append(feature[0])
    col = pd.DataFrame({'chunk %d'%chunk: a })
    table =  pd.concat([table, col],axis = 1)


table.index=['R2_train', 'RMSE_train', 'R2_test', 'RMSE_test','b','Page Popularity','Page Checkins',
             'Page talking about','extra_0','extra_1','extra_2','extra_3','extra_4','extra_5','extra_6',
             'extra_7','extra_8','extra_9','extra_10','extra_11','extra_12','extra_13','extra_14','extra_15',
             'extra_16','extra_17','extra_18','extra_19','extra_20','extra_21','extra_22','extra_23','extra_24',
             'CC1','CC2','CC3','CC4','CC5','Base Time','Post Length','Post Share Count','H Local','Page Category', 
             'Post Promotion Status','published_weekday_0','published_weekday_1','published_weekday_2',
             'published_weekday_3', 'published_weekday_4','published_weekday_5','published_weekday_6',
             'base_weekday_0','base_weekday_1','base_weekday_2','base_weekday_3','base_weekday_4',
             'base_weekday_5','base_weekday_6']
table.index.name = 'Features'
table =  pd.concat([table, pd.DataFrame({'E': table.mean(axis = 1) })],axis = 1)
table =  pd.concat([table, pd.DataFrame({'STD': table.std(axis = 1) })],axis = 1)
table.to_csv('table.csv')   

CHUNK #  1
R2 for train:  0.330076294116 

RMSE for train:  28.843231697267996 

R2 for test:  0.269729266175 

RMSE for test:  31.186284710647545 

CHUNK #  2
R2 for train:  0.303070861247 

RMSE for train:  29.92592889313813 

R2 for test:  0.37268394539 

RMSE for test:  26.9656164022526 

CHUNK #  3
R2 for train:  0.308057121781 

RMSE for train:  29.314441254270342 

R2 for test:  0.330620657924 

RMSE for test:  29.853893519645275 

CHUNK #  4
R2 for train:  0.33304796984 

RMSE for train:  28.58570658518897 

R2 for test:  0.264326864091 

RMSE for test:  32.07492963556872 

CHUNK #  5
R2 for train:  0.317937681945 

RMSE for train:  29.837172129652473 

R2 for test:  0.313715696574 

RMSE for test:  27.200488061215125 



In [75]:
table

,chunk 1,chunk 2,chunk 3,chunk 4,chunk 5,E,STD
Features,,,,,,,
R2_train,0.330076,0.303071,0.308057,0.333048,0.317938,0.318438,0.011773
RMSE_train,28.843232,29.925929,29.314441,28.585707,29.837172,29.301296,0.529036
R2_test,0.269729,0.372684,0.330621,0.264327,0.313716,0.310215,0.040189
RMSE_test,31.186285,26.965616,29.853894,32.074930,27.200488,29.456242,2.063993
b,4.301475,4.348245,4.330156,4.341225,4.363566,4.336933,0.020779
Page Popularity,-0.065343,-0.134205,-0.538271,0.023551,-0.090004,-0.160854,0.195602
Page Checkins,-0.588408,-0.674146,-0.609190,-0.579901,-0.684505,-0.627230,0.043713
Page talking about,-1.026365,-0.778213,-0.631900,-1.335379,-1.044728,-0.963317,0.242130
extra_0,-0.033429,-0.187745,-0.242326,-0.231111,-0.200027,-0.178928,0.075413


In [35]:
nfolds = 5
indexes = np.arange(10)
indexes = indexes.reshape(2,5)
li =[] 
for i in range(nfolds):
    li.append(indexes[:,i].tolist())
Xfold =[]
Yfold =[]

    

In [36]:
l=[]
for i in range(5):
    if i!=0:
        l = l+li[i]
    
    print(l)
   
    

[]
[1, 6]
[1, 6, 2, 7]
[1, 6, 2, 7, 3, 8]
[1, 6, 2, 7, 3, 8, 4, 9]
